<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [3]:
!pip3 install beautifulsoup4
!pip3 install requests

In [4]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [7]:
import requests

# URL provided (replace with actual URL if different)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform GET request
response = requests.get(static_url)

# Display status code to confirm success (200 means OK)
print("Status Code:", response.status_code)

Status Code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [8]:
from bs4 import BeautifulSoup

# Create BeautifulSoup object from the HTML content of the response
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [9]:
# Print the page title to verify the BeautifulSoup object
print("Page Title:", soup.title.string)

Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [ ]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`


Starting from the third table is our target table contains the actual launch records.


In [10]:
# Find all <table> elements in the HTML
html_tables = soup.find_all('table')

# Print how many tables were found
print("Number of tables found:", len(html_tables))


Number of tables found: 25


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [29]:
# Assume you already have the helper function extract_column_from_header()
# and the first launch table from html_tables[0]

# Select the first table
first_launch_table = html_tables[0]

# Initialize empty list to store column names
column_names = []

# Find all <th> elements from the header row
table_headers = first_launch_table.find_all('th')

# Iterate through each <th> element
for th in table_headers:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Display the extracted column names
print("Extracted column names:")
print(column_names)


Extracted column names:
[]


Check the extracted column names


In [21]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [31]:
# Get all rows from the table
table_rows = first_launch_table.find_all('tr')[1:]  # skip header row

# Loop through each row
for row in table_rows:
    cells = row.find_all('td')
    if len(cells) == 0:
        continue  # skip empty rows
    
    try:
        # Extract and clean values
        flight_no = cells[0].text.strip()
        date_time = cells[1].text.strip().split(' ')
        version_booster = cells[2].text.strip()
        launch_site = cells[3].text.strip()
        payload = cells[4].text.strip()
        payload_mass = cells[5].text.strip()
        orbit = cells[6].text.strip()
        customer = cells[7].text.strip()
        launch_outcome = cells[8].text.strip()
        booster_landing = cells[9].text.strip()
        
        # Clean up date/time
        date = date_time[0]
        time = date_time[1] if len(date_time) > 1 else '00:00'

        # Append to dict
        launch_dict['Flight No.'].append(flight_no)
        launch_dict['Date'].append(date)
        launch_dict['Time'].append(time)
        launch_dict['Version Booster'].append(version_booster)
        launch_dict['Launch site'].append(launch_site)
        launch_dict['Payload'].append(payload)
        launch_dict['Payload mass'].append(payload_mass)
        launch_dict['Orbit'].append(orbit)
        launch_dict['Customer'].append(customer)
        launch_dict['Launch outcome'].append(launch_outcome)
        launch_dict['Booster landing'].append(booster_landing)

    except IndexError:
        # Skip row if unexpected format
        continue


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [28]:
extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table row 
    for rows in table.find_all("tr"):
        # Check to see if first table heading is a number corresponding to a launch number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get table element 
        row = rows.find_all('td')
        
        # If it is a launch row, save cells in the dictionary 
        if flag:
            extracted_row += 1

            # Flight Number
            launch_dict['flight No.'].append(flight_no)

            # Date and Time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')  # remove trailing comma
            time = datatimelist[1]
            launch_dict['Date'].append(date)
            launch_dict['Time'].append(time)

            # Booster Version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string
            launch_dict['Customer'].append(customer)

            # Launch Outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster Landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


KeyError: 'flight No.'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [37]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })
import requests
import pandas as pd

# Step 1: Send GET request to SpaceX API
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

# Step 2: Convert JSON response to a DataFrame
data = response.json()
df = pd.json_normalize(data)

# Step 3: Display the first value of the `static_fire_date_utc` column
first_date = df.loc[0, 'static_fire_date_utc']
print("static_fire_date_utc (row 0):", first_date)

# Step 4: Extract the year
if first_date:
    year = pd.to_datetime(first_date).year
    print("Year:", year)
else:
    print("No static fire date available in the first row.")

static_fire_date_utc (row 0): 2006-03-17T00:00:00.000Z
Year: 2006


In [40]:
import requests
import pandas as pd

# Step 1: Get launch data
launches_url = "https://api.spacexdata.com/v4/launches"
launches_response = requests.get(launches_url)
launches = launches_response.json()
launches_df = pd.json_normalize(launches)

# Step 2: Get rocket metadata
rockets_url = "https://api.spacexdata.com/v4/rockets"
rockets_response = requests.get(rockets_url)
rockets = rockets_response.json()
rockets_df = pd.json_normalize(rockets)

# 🔍 Check column names to debug
print("Rockets columns:", rockets_df.columns)

# Step 3: Merge rocket name into launch dataframe
# We use rockets_df['id'] and rockets_df['name']
merged_df = launches_df.merge(
    rockets_df[['id', 'name']], 
    left_on='rocket', 
    right_on='id', 
    how='left',
    suffixes=('', '_rocket')
)

# Step 4: Filter Falcon 9 launches
falcon9_launches = merged_df[merged_df['name'] == 'Falcon 9']

# Step 5: Print result
print("Number of Falcon 9 launches:", len(falcon9_launches))


Rockets columns: Index(['payload_weights', 'flickr_images', 'name', 'type', 'active', 'stages',
       'boosters', 'cost_per_launch', 'success_rate_pct', 'first_flight',
       'country', 'company', 'wikipedia', 'description', 'id', 'height.meters',
       'height.feet', 'diameter.meters', 'diameter.feet', 'mass.kg', 'mass.lb',
       'first_stage.thrust_sea_level.kN', 'first_stage.thrust_sea_level.lbf',
       'first_stage.thrust_vacuum.kN', 'first_stage.thrust_vacuum.lbf',
       'first_stage.reusable', 'first_stage.engines',
       'first_stage.fuel_amount_tons', 'first_stage.burn_time_sec',
       'second_stage.thrust.kN', 'second_stage.thrust.lbf',
       'second_stage.payloads.composite_fairing.height.meters',
       'second_stage.payloads.composite_fairing.height.feet',
       'second_stage.payloads.composite_fairing.diameter.meters',
       'second_stage.payloads.composite_fairing.diameter.feet',
       'second_stage.payloads.option_1', 'second_stage.reusable',
       'second_s

In [44]:
import pandas as pd
import requests

# Step 1: Request SpaceX launch data
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
data = response.json()

# Step 2: Normalize to DataFrame
df = pd.json_normalize(data)

# Step 3: Extract the 'landpad' values from nested 'cores' list
df['landingPad'] = df['cores'].apply(
    lambda x: x[0].get('landpad') if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else None
)

# Step 4: Count missing values
missing_count = df['landingPad'].isnull().sum()
print(f"❓ Missing values in 'landingPad': {missing_count}")



❓ Missing values in 'landingPad': 54


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
